In this Note book i try two different model, lightgbmClassifier and LinearSVC which gives a score of 0.042.., here i used kfold from sklearn, the dataset is split to 10,20,30k fold for linear svc to see if it will improve but the 10k-fold prove to be the best followed by 20k

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import re
import lightgbm as lgbm
from sklearn import model_selection

In [ ]:
lgbm.LGBMClassifier

In [3]:
test_df  = pd.read_csv("test_clean.csv")
train_df = pd.read_csv("train_clean.csv")

In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [5]:
train_df['Text'] = train_df['Text'].map(lambda com : clean_text(com))
test_df['Text'] = test_df['Text'].map(lambda com : clean_text(com))

In [6]:
labels  = train_df.columns[3:]

In [7]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

## Preprocess text
I used tf-idf to preprocess the dataset and use a range from uni to trigram to extract text features,

In [8]:
Text = 'Text'
n = train_df.shape[0]
vec = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [9]:
trn_term_doc = vec.fit_transform(train_df[Text])
test_term_doc = vec.transform(test_df[Text])

In [2]:
##params for lgbm
params = {
    'objective' :'binary',
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss'
}

In [30]:
kfold = model_selection.KFold(n_splits=30)
preds = np.zeros((len(test_df),len(labels)))

## NBSVM
 NBSVM was introduced by Sida Wang and Chris Manning in the paper [Baselines and Bigrams: Simple, Good Sentiment and Topic Classiﬁcation](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf). 
 the function pr(y_i,y) help the model to train with  naive bayes features, i.e the tfidf vectors ae turn to naive bayes features which is the probability of the 1 and 0 in the label

In [31]:
def pr(y_i, y):
    p = trn_term_doc[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [32]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

### so the model is train one by one for each label

In [33]:
for i, category in enumerate(labels):
    print("training...",category)
    y = train_df[category].values
    r = np.log(pr(1,y) / pr(0,y))
    pred = np.zeros((len(test_df),))
#     if i >0:
#         break
    for train,test in kfold.split(trn_term_doc):
        
        
        X_train = trn_term_doc[train].multiply(r)
        X_val = trn_term_doc[test].multiply(r)
        y_train = y[train]
        y_val = y[test]
        
#         d_train = lgbm.Dataset(X_train,y_train)
#         d_valid = lgbm.Dataset(X_val,y_val)
        
#         clf = lgbm.train(params,d_train,1000,valid_sets=[d_valid],verbose_eval=50,early_stopping_rounds=50)
        
        clf = LinearSVC()
        clf.fit(X_train,y_train)
        prediction = clf.predict(X_val)
        print('Test accuracy is {}'.format(accuracy_score(y_val, prediction)))
        
        pred += clf.predict(test_term_doc.multiply(r))
        

                
    n_pred = (pred - pred.min())/(pred.max() - pred.min())
    for j in range(0,998):
        if n_pred[j]>=.5:
            n_pred[j]=1
        else:
            n_pred[j]=0
    print(n_pred.shape)
    preds[:,i] = n_pred
        
    

training... 3.1.1
Test accuracy is 0.95
Test accuracy is 0.95
Test accuracy is 1.0
Test accuracy is 0.9
Test accuracy is 0.96
Test accuracy is 0.93
Test accuracy is 0.97
Test accuracy is 0.97
Test accuracy is 0.94
Test accuracy is 0.97
Test accuracy is 0.97
Test accuracy is 0.98
Test accuracy is 0.97
Test accuracy is 0.97
Test accuracy is 0.94
Test accuracy is 0.98
Test accuracy is 0.98
Test accuracy is 0.95
Test accuracy is 0.96
Test accuracy is 0.91
Test accuracy is 0.98
Test accuracy is 0.96
Test accuracy is 0.99
Test accuracy is 0.9
Test accuracy is 0.98
Test accuracy is 0.9595959595959596
Test accuracy is 0.9393939393939394
Test accuracy is 0.9393939393939394
Test accuracy is 0.9696969696969697
Test accuracy is 0.98989898989899
(998,)
training... 3.1.2
Test accuracy is 0.94
Test accuracy is 0.97
Test accuracy is 0.95
Test accuracy is 0.97
Test accuracy is 0.98
Test accuracy is 0.99
Test accuracy is 1.0
Test accuracy is 0.96
Test accuracy is 0.97
Test accuracy is 0.96
Test accuracy

In [35]:
submid = pd.DataFrame({'ID': test_df["Unique ID"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = labels)], axis=1)
submission.to_csv('submissionnsvc3_20.csv', index=False,float_format="%.f")